In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
path = '/content/drive/MyDrive/final_project_itc/data.npy'
data = np.load(path, allow_pickle = True)

In [4]:
df = pd.DataFrame(data, columns = ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'])

In [5]:
df = df.loc[:,['ProductId', 'Text', 'Score']]

In [6]:
df.loc[df.Score == 1,'Score'] = 0
df.loc[df.Score == 2,'Score'] = 0
df.loc[df.Score == 3,'Score'] = 1
df.loc[df.Score == 4,'Score'] = 2
df.loc[df.Score == 5,'Score'] = 2

#### Split the dataset into train val and test using 100k samples

In [7]:
from sklearn.model_selection import GroupShuffleSplit
splitter_temp = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 42)
split_temp = splitter_temp.split(df[:100000], groups=df[:100000]['ProductId'])
train_inds, temp_inds = next(split_temp)

train = df.iloc[train_inds]
temp = df.iloc[temp_inds]



In [8]:
train.Score.value_counts(normalize = True)

2    0.775211
0    0.146850
1    0.077940
Name: Score, dtype: float64

In [9]:
splitter_val = GroupShuffleSplit(test_size=.50, n_splits=1, random_state = 42)
split_val = splitter_val.split(temp, groups=temp['ProductId'])
val_inds, test_inds = next(split_val)

val = temp.iloc[val_inds]
test = temp.iloc[test_inds]

In [10]:
val.Score.value_counts(normalize = True)

2    0.761565
0    0.152083
1    0.086352
Name: Score, dtype: float64

In [11]:
test.Score.value_counts(normalize = True)

2    0.766295
0    0.151440
1    0.082265
Name: Score, dtype: float64

In [12]:
train.shape,val.shape,test.shape

((59251, 3), (21725, 3), (19024, 3))

In [13]:
X_train = train.drop(columns = 'Score')
y_train = train.Score

X_val = val.drop(columns = 'Score')
y_val = val.Score

X_test = test.drop(columns = 'Score')
y_test = test.Score

#### BERT transformer

In [14]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

##### Bert classification

In [15]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
base_model = 'bert-base-cased'
learning_rate = 2e-5
max_length = 64
batch_size = 32
epochs = 5
id2label = {k:k for k in range(3)}
label2id = {k:k for k in range(3)}

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForSequenceClassification.from_pretrained(base_model,id2label = id2label, label2id = label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
X_train_pytorch = X_train.iloc[:1000] ## when i rerun i will have to change bc i deleted productid from X_train

In [17]:
del X_train_pytorch['ProductId']

In [18]:
X_train_pytorch['label'] = y_train.iloc[:1000]

<ipython-input-18-1b5c0afe216c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_pytorch['label'] = y_train.iloc[:1000]


In [19]:
X_val_pytorch = X_val.iloc[:1000]
del X_val_pytorch['ProductId']
X_val_pytorch['label'] = y_val.iloc[:1000]

<ipython-input-19-eb5896239af3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_pytorch['label'] = y_val.iloc[:1000]


In [20]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [21]:
from datasets import Dataset
ds_train = Dataset.from_pandas(X_train_pytorch)
ds_val = Dataset.from_pandas(X_val_pytorch)

In [22]:
def preprocess_function(examples):
    return tokenizer(examples["Text"], truncation=True, max_length=326, padding = 'max_length')

In [23]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-23-e1ee7db0da93>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [24]:
output_dir = "/content/drive/MyDrive/"

In [25]:
from transformers import TrainingArguments
output_dir = output_dir
training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

In [26]:
ds_train_tokenized = ds_train.map(preprocess_function, remove_columns = ['Text'])
ds_val_tokenized = ds_val.map(preprocess_function,remove_columns = ['Text'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train_tokenized,
    eval_dataset=ds_val_tokenized,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.548130,0.766000
2,No log,0.422010,0.853000
3,No log,0.414872,0.861000
4,No log,0.467130,0.855000
5,No log,0.414338,0.859000


TrainOutput(global_step=160, training_loss=0.3871971607208252, metrics={'train_runtime': 299.3456, 'train_samples_per_second': 16.703, 'train_steps_per_second': 0.534, 'total_flos': 837646232220000.0, 'train_loss': 0.3871971607208252, 'epoch': 5.0})

In [28]:
trainer.evaluate() # best accuracy we got on the val set.

{'eval_loss': 0.4148719608783722,
 'eval_accuracy': 0.861,
 'eval_runtime': 6.7825,
 'eval_samples_per_second': 147.439,
 'eval_steps_per_second': 4.718,
 'epoch': 5.0}

In [29]:
X_test_pytorch = X_test.iloc[:1000]
del X_test_pytorch['ProductId']

In [30]:
X_test_pytorch['label'] = y_test.iloc[:1000]

<ipython-input-30-f39a701bddb1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_pytorch['label'] = y_test.iloc[:1000]


In [31]:
ds_test = Dataset.from_pandas(X_test_pytorch)

In [32]:
ds_test_tokenized = ds_test.map(preprocess_function,remove_columns = ['Text'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [33]:
trainer.eval_dataset = ds_test_tokenized

In [34]:
eval_results = trainer.evaluate()

In [35]:
eval_results

{'eval_loss': 0.4091423451900482,
 'eval_accuracy': 0.857,
 'eval_runtime': 6.5633,
 'eval_samples_per_second': 152.362,
 'eval_steps_per_second': 4.876,
 'epoch': 5.0}

In [36]:
predictions = trainer.predict(ds_test_tokenized)

In [37]:
predicted_labels = predictions.predictions

In [38]:
predicted_labels_int = predicted_labels.argmax(axis = 1)

In [59]:
from sklearn.metrics import classification_report
print(classification_report(predicted_labels_int,y_test.iloc[:1000].astype(int).values))

              precision    recall  f1-score   support

           0       0.71      0.70      0.70       141
           1       0.00      0.00      0.00         1
           2       0.97      0.88      0.93       858

    accuracy                           0.86      1000
   macro avg       0.56      0.53      0.54      1000
weighted avg       0.94      0.86      0.89      1000



##### Try our model on input sentences

In [65]:
import torch
input_texts = ["Flavor is amazing"]

# Encode the text
encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=256, return_tensors="pt").to("cuda")

# Call the model to predict under the format of logits of 5 classes
logits = model(**encoded).logits

# Get the class
torch.argmax(logits, axis=1)

tensor([2], device='cuda:0')